In [1]:
from nltk.tokenize import TweetTokenizer
from nltk.stem.snowball import RussianStemmer
import pandas as pd
import re
import tensorflow as tf
import numpy as np
import sys
import vk
import telebot


ImportError: Traceback (most recent call last):
  File "c:\users\user\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 18, in swig_import_helper
    return importlib.import_module(mname)
  File "c:\users\user\appdata\local\programs\python\python36\lib\importlib\__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 994, in _gcd_import
  File "<frozen importlib._bootstrap>", line 971, in _find_and_load
  File "<frozen importlib._bootstrap>", line 955, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 658, in _load_unlocked
  File "<frozen importlib._bootstrap>", line 571, in module_from_spec
  File "<frozen importlib._bootstrap_external>", line 922, in create_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
ImportError: DLL load failed: Не найдена указанная процедура.

Duri

In [ ]:
t = TweetTokenizer()
stemer = RussianStemmer()
regex = re.compile('[^а-яА-Я ]')

In [ ]:
length_of_the_dictionary = 6 #stem words
tweets_col_number = 3
negative_tweets = pd.read_csv(
    'negative.csv', header=None, delimiter=';')[[tweets_col_number]]
positive_tweets = pd.read_csv(
    'positive.csv', header=None, delimiter=';')[[tweets_col_number]]

In [ ]:
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
w = tf.get_variable("Weights", [310000,1], dtype=tf.float64, initializer=tf.glorot_uniform_initializer(
    seed=None,
    dtype=tf.float64
))
sess.run (tf.global_variables_initializer())
dictionary = []
expected = tf.placeholder(tf.float64)
x = tf.placeholder(tf.float64)
predict = tf.sigmoid(tf.matmul(x,w))
loss = tf.pow(predict - expected,2)
Optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

In [ ]:
for i,tweet in positive_tweets.iterrows():
    for token in t.tokenize(tweet[3]):
        vector = np.zeros((1,310000))
        stem = stemer.stem(token)
        if stem not in dictionary: dictionary.append(stem)
        vector[0,dictionary.index(stem)] = 1.0
        sess.run([loss,Optimizer],feed_dict={x: vector,expected: 1.0})
    
    
    if len(dictionary)>=400:break

print ('Positive is done !')


for i,tweet in negative_tweets.iterrows():
    for token in t.tokenize(tweet[3]):
        vector = np.zeros((1,310000))
        stem = stemer.stem(token)
        if stem not in dictionary: dictionary.append(stem)
        vector[0,dictionary.index(stem)] = 1.0
        sess.run([loss,Optimizer],feed_dict={x: vector,expected: 0.0})
    if len(dictionary)>=800:break     
    
print ('Negative is done !')
    


In [ ]:
vector = np.zeros((1,310000))   
for i in t.tokenize('я мама школота  '):
    
    
    print (stemer.stem(i))
    print (dictionary.index(stemer.stem(i)))
    vector[0,dictionary.index(stemer.stem(i))] = 1.0
print (sess.run(predict,feed_dict={x: vector}))

In [ ]:
bot = telebot.TeleBot('519802622:AAERN18WJLVog4eCLhl6lnB2htkJM8urV-4')
@bot.message_handler(content_types=["text"])
def repeat_all_messages(messag):
    vector = np.zeros((1,310000))   
    for i in t.tokenize(messag.text):
        
        vector[0,dictionary.index(stemer.stem(i))] = 1.0
        
    bot.send_message(messag.chat.id,sess.run(predict,feed_dict={x: vector}) )    
   
    




In [ ]:
bot.polling(none_stop=True)

In [2]:
# Creates a graph.
with tf.device('/device:GPU:0'):
        
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
    c = tf.matmul(a, b)
# Creates a session with log_device_placement set to True.
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# Runs the op.
print(sess.run(c))

InvalidArgumentError: Cannot assign a device for operation 'MatMul': Operation was explicitly assigned to /device:GPU:0 but available devices are [ /job:localhost/replica:0/task:0/device:CPU:0 ]. Make sure the device specification refers to a valid device.
	 [[Node: MatMul = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, _device="/device:GPU:0"](a, b)]]

Caused by op 'MatMul', defined at:
  File "c:\users\user\appdata\local\programs\python\python36\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\users\user\appdata\local\programs\python\python36\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "c:\users\user\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "c:\users\user\appdata\local\programs\python\python36\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "c:\users\user\app